# Web Scrapping Assignment 3

In [1]:
#importing the required libraries
import pandas as pd
import selenium
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

##### 1. Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [186]:
#loading the drivers and URL

driver=webdriver.Chrome('chromedriver.exe')
driver.get('http://amazon.in/')


# Creating the inpiut for the product name to be entered
product=input("Enter the product name: ")

#updating the input details in seracg=h bar
driver.find_element_by_id("twotabsearchtextbox").send_keys(product)
driver.find_element_by_id("nav-search-submit-button").click()

Enter the product name: Laptop


##### 2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a dataframe and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [187]:
#to update the url of all products from 3 pages
urls=[]
for i in range(3):
    for j in driver.find_elements_by_xpath("//div[@class='a-section a-spacing-medium']//h2"):
        urls.append(j.find_element_by_xpath(".//a").get_attribute('href'))
    try:
        next_page=driver.find_element_by_xpath("//div[@class='a-text-center']/ul/li[@class='a-last']/a").get_attribute('href')
        driver.get(next_page)
        driver.refresh()   
    except:
        break


In [188]:
#creating the empty list
brand=[]
name=[]
rating=[]
num_of_rating=[]
price=[]
return_exc=[]
exp_del=[]
avail=[]
pro_url=[]

#updating all the prodcut urls
for i in urls:
    driver.get(i)
    pro_url.append(i)
    time.sleep(2)
    
#Scrapping the brand name
    try:
        brand.append(driver.find_element_by_id("productTitle").text.split(' ')[0])
        
    except NoSuchElementException:
        brand.append("-")
            
#Scrapping the prodcut name
    try:
        name.append(driver.find_element_by_id("productTitle").text.split(' ')[1:]) 
    except NoSuchElementException:
        name.append("-")
            
#Scrapping the product rating
    try:
        rating.append(driver.find_element_by_xpath('//*[@id="a-popover-content-8"]/div/div/div/div[1]/span').text)
    except NoSuchElementException:
        rating.append("-")

#Scrapping the number of rating
    try:
        num_of_rating.append(driver.find_element_by_xpath('//*[@id="acrCustomerReviewText"]').text)
    except NoSuchElementException:
        num_of_rating.append("-")
    
#Scrapping the price
    try: 
        try: price.append(driver.find_element_by_id("priceblock_saleprice").text) 
        except:
            try: price.append(driver.find_element_by_id("priceblock_dealprice").text)
            except: price.append(driver.find_element_by_id("priceblock_ourprice").text)
    except: price.append('-')
            
#scrapping for return/exchange
    try:
        return_exc.append(driver.find_element_by_xpath('//*[@id="RETURNS_POLICY"]/span/div[2]/a').text)
    except NoSuchElementException:
        return_exc.append("-")
            
#scrapping for expected delivery
    try:
        exp_del.append(driver.find_element_by_xpath('//*[@id="ddmDeliveryMessage"]/b').text)
    except NoSuchElementException:
        exp_del.append("-")

#scrapping for availability
    try:
        avail.append(driver.find_element_by_id("availability").text)
    except NoSuchElementException:
        avail.append("-")
            

In [192]:
#creating the DataFrame
Product={'Brand':brand,'Name':name,'Rating':rating,'Number of rating':num_of_rating,'Price':price,
        'Return/Exchange':return_exc,'Expected Delivery':exp_del,'Availability':avail,'URL':pro_url}
Product=pd.DataFrame(data=Product)
Product

,Brand,Name,Rating,Number of rating,Price,Return/Exchange,Expected Delivery,Availability,URL
0,Lenovo,"[Legion, 5, 10th, Gen, Intel, Core, i5, 15.6, ...",-,206 ratings,"₹ 74,990.00",7 Days Replacement,"Tuesday, July 6",In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
1,ASUS,"[VivoBook, 15, (2020),, 39.6, cm, HD,, Dual, C...",-,607 ratings,"₹ 28,298.00",7 Days Replacement,"Friday, July 9","In stock on July 7, 2021.\nOrder it now.",https://www.amazon.in/ASUS-VivoBook-Integrated...
2,HP,"[15, Entry, Level, 15.6-inch, (39.62, cms), HD...",-,"1,181 ratings","₹ 25,990.00",7 Days Replacement,"Tuesday, July 13","In stock on July 11, 2021.\nOrder it now.",https://www.amazon.in/HP-15-6-inch-Laptop-Wind...
3,HP,"[15, Intel, Pentium, Gold, 6405U, Processor, E...",-,432 ratings,"₹ 27,990.00",7 Days Replacement,"Tuesday, July 6",In stock.,https://www.amazon.in/HP-Pentium-Processor-15-...
4,(Renewed),"[Dell, Latitude, 15.6, inch, (39.62, cm), HD, ...",-,-,"₹ 31,999.00",7 Days Replacement,"Friday, July 9",In stock.,https://www.amazon.in/Dell-Latitude-Business-B...
5,Lenovo,"[Ideapad, Slim, 3, 15.6"", HD, Thin, and, Light...",-,8 ratings,-,7 Days Replacement,-,,https://www.amazon.in/Lenovo-Ideapad-Laptop-Wi...
6,HP,"[15, (2021), Thin, &, Light, Ryzen, 3-3250, La...",-,551 ratings,"₹ 41,990.00",7 Days Replacement,"Tuesday, July 6",In stock.,https://www.amazon.in/HP-3-3250-Laptop-Windows...
7,Lenovo,"[ThinkPad, E15, (2021), Intel, Core, i3, 11th,...",-,-,"₹ 46,389.00",7 Days Replacement,Tomorrow,In stock.,https://www.amazon.in/Lenovo-ThinkPad-Extended...
8,(Renewed),"[HP, 15, (2021), Thin, &, Light, Ryzen, 3-3250...",-,-,"₹ 33,799.00",7 Days Replacement,"Friday, July 9",In stock.,https://www.amazon.in/Renewed-HP-3-3250-Window...
9,(Renewed),"[Fujitsu, Intel, Core, i5, 3340M, 15.6-Inch, (...",-,74 ratings,"₹ 19,990.00",7 Days Replacement,"Saturday, July 10",In stock.,https://www.amazon.in/Renewed-Fujitsu-15-6-Inc...


##### 3. Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [2]:
# We will create empty list in which we will update the required data
fruits=[]
cars=[]
machine=[]

#creating the list for the name of the items we want the images
Name=['fruits','cars','machine']

#loading the drivers and url
driver=webdriver.Chrome("chromedriver.exe")
driver.maximize_window()
wait = WebDriverWait(driver,15)
driver.get("https://www.google.co.in/imghp?hl=en&ogbl")

for i in Name:
    if i=='fruits':
        driver.find_element_by_xpath("//input[@class='gLFyf gsfi']").send_keys(i) #entering the item for search
        driver.find_element_by_xpath("//button[@class='Tg7LZd']").click()

#creating the instance to scroll down the page for images to load.
        SCROLL_PAUSE_TIME = 2
        CYCLES = 10

        html = driver.find_element_by_tag_name('html')
        html.send_keys(Keys.PAGE_DOWN)  
        html.send_keys(Keys.PAGE_DOWN)  
        time.sleep(SCROLL_PAUSE_TIME * 3)

        for j in range(CYCLES):
            html.send_keys(Keys.END)
            time.sleep(SCROLL_PAUSE_TIME)

#scrapping for fruits            
    fruit=driver.find_elements_by_xpath("//a[@class='VFACy kGQAp sMi44c lNHeqe WGvvNb']")
    for k in fruit:
        fruits.append(k.get_attribute('href'))
    html.send_keys(Keys.HOME)
    driver.find_element_by_xpath("//input[@class='og3lId']").clear() #clearing the data in searc bar

#scrapping for cars
    if i=='cars':
        driver.find_element_by_xpath("//input[@class='og3lId']").send_keys(i)
        driver.find_element_by_xpath("//button[@class='rCGXm']").click()
        
        SCROLL_PAUSE_TIME = 2
        CYCLES = 10

        html = driver.find_element_by_tag_name('html')
        html.send_keys(Keys.PAGE_DOWN)  
        html.send_keys(Keys.PAGE_DOWN)  
        time.sleep(SCROLL_PAUSE_TIME * 3)

        for j in range(CYCLES):
            html.send_keys(Keys.END)
            time.sleep(SCROLL_PAUSE_TIME)
    car=driver.find_elements_by_xpath("//a[@class='VFACy kGQAp sMi44c lNHeqe WGvvNb']")
    for k in car:
        cars.append(k.get_attribute('href'))
    html.send_keys(Keys.HOME)
    driver.find_element_by_xpath("//input[@class='og3lId']").clear()

#scrapping for machine

    if i=='machine':
        driver.find_element_by_xpath("//input[@class='og3lId']").send_keys(i)
        driver.find_element_by_xpath("//button[@class='rCGXm']").click()
        
        SCROLL_PAUSE_TIME = 2
        CYCLES = 10

        html = driver.find_element_by_tag_name('html')
        html.send_keys(Keys.PAGE_DOWN)  
        html.send_keys(Keys.PAGE_DOWN)  
        time.sleep(SCROLL_PAUSE_TIME * 3)

        for j in range(CYCLES):
            html.send_keys(Keys.END)
            time.sleep(SCROLL_PAUSE_TIME)
    mac=driver.find_elements_by_xpath("//a[@class='VFACy kGQAp sMi44c lNHeqe WGvvNb']")
    for k in mac:
        machine.append(k.get_attribute('href'))

In [6]:
#creating the dataframe
Google={'Fruits':fruits,'Cars':cars,'Machine':machine}
Google=pd.DataFrame(data=Google)
Google[:100]

,Fruits,Cars,Machine
0,https://www.pinterest.com/pin/649081365039047796/,https://www.pinterest.com/pin/649081365039047796/,https://www.pinterest.com/pin/649081365039047796/
1,https://www.youtube.com/watch?v=PY2OxP_jMZ0,https://www.youtube.com/watch?v=PY2OxP_jMZ0,https://www.youtube.com/watch?v=PY2OxP_jMZ0
2,https://www.goodhousekeeping.com/health/diet-n...,https://www.goodhousekeeping.com/health/diet-n...,https://www.goodhousekeeping.com/health/diet-n...
3,https://www.healthyeating.org/nutrition-topics...,https://www.healthyeating.org/nutrition-topics...,https://www.healthyeating.org/nutrition-topics...
4,https://www.pinterest.com/pin/641340803167435998/,https://www.pinterest.com/pin/641340803167435998/,https://www.pinterest.com/pin/641340803167435998/
...,...,...,...
95,https://www.berries.com/blog/what-fruits-are-i...,https://www.motortrend.com/features-collection...,https://www.motortrend.com/features-collection...
96,https://www.popularmechanics.com/cars/a3482425...,https://www.popularmechanics.com/cars/a3482425...,https://en.wikipedia.org/wiki/Machine
97,https://www.wired.com/story/jd-power-korean-ca...,https://www.wired.com/story/jd-power-korean-ca...,https://www.macmillandictionaryblog.com/machine
98,https://luxe.digital/lifestyle/cars/fastest-cars/,https://luxe.digital/lifestyle/cars/fastest-cars/,https://www.ies.co.uk/news/10-iconic-machines-...


##### 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. In case if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [43]:
#loading the driver and the url
driver=webdriver.Chrome('chromedriver.exe')
time.sleep(2)
url=('https://www.flipkart.com/')
driver.get(url)
time.sleep(2)
driver.find_element_by_xpath("//button[@class='_2KpZ6l _2doB4z']").click()
time.sleep(2)

In [59]:
#creating the user input to search for mobiles
user_inp= input('Enter the product you want to search: ')
driver.find_element_by_xpath("//div[@class='_3OO5Xc']/input").send_keys(user_inp)
time.sleep(2)
driver.find_element_by_xpath("//button[@class='L0Z3Pu']").click()
time.sleep(2)

Enter the product you want to search: Smartphone


In [60]:
#creating the empty list
Brand=[] 
Smartphone_name=[]
Colour=[]
RAM=[] 
Storage=[]
Primary_Camera=[]
Secondary_Camera=[]
Display_Size=[]
Display_Resolution=[]
Processor=[]
Processor_Cores=[]
Battery_Capacity=[]
Price=[]
Product_url=[]

In [ ]:
#clicking on the product link
product_links=driver.find_elements_by_xpath("//a[@class='_1fQZEK']")

#scrapping the product link
for i in product_links:
    Product_url.append(i.get_attribute('href'))
for i in Product_url:
    driver.get(i)
    driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]').click()
#scrapping the Brand name
    try:
        brand=driver.find_element_by_xpath('//span[@class="B_NuCI"]').text.split()
        Brand.append(brand[0])
    except:
        Brand.append('-')
#scrapping other details
    a= driver.find_elements_by_xpath("//li[@class='_21lJbe']")
    b=driver.find_elements_by_xpath("//td[@class='_1hKmbr col col-3-12']")
    for i in range(0,len(b)):
        if b[i].text=='Model Name':
            Smartphone_name.append(a[i].text)        
        if b[i].text=='Color':
            Colour.append(a[i].text)
        if b[i].text=='RAM':
            RAM.append(a[i].text)
        if b[i].text=='Internal Storage':
            Storage.append(a[i].text)
        if b[i].text=='Primary Camera':
            Primary_Camera.append(a[i].text)
        if b[i].text=='Secondary Camera':
            Secondary_Camera.append(a[i].text)                                  
        if b[i].text=='Display Size':
            Display_Size.append(a[i].text)
        if b[i].text=='Resolution':
            Display_Resolution.append(a[i].text)
        if b[i].text=='Processor Type':
            Processor.append(a[i].text)            
        if b[i].text=='Processor Core':
            Processor_Cores.append(a[i].text)
        if b[i].text=='Battery Capacity':
            Battery_Capacity.append(a[i].text)
#scrapping the prices
    try:
        price=driver.find_element_by_xpath('//div[@class="_30jeq3 _16Jk6d"]')
        Price.append(price.text)
    except:
        Price.append('-')

In [78]:
#creating the dataframe
Details={'Brand Name':Brand,'Smartphone Name':Smartphone_name,'Color':Colour,'RAM':RAM,'Storage(ROM)':Storage,
        'Primary Camera':Primary_Camera,'Secondary Camera':Secondary_Camera,'Display Size':Display_Size,
         'Display Resolution':Display_Resolution,'Processor':Processor,'Processor Cores':Processor_Cores,
        'Battery Capacity':Battery_Capacity,'Price':Price}

Details=pd.DataFrame(data=Details)
Details

,Brand Name,Smartphone Name,Color,RAM,Storage(ROM),Primary Camera,Secondary Camera,Display Size,Display Resolution,Processor,Processor Cores,Battery Capacity,Price
0,Infinix,Hot 10 Play,Morandi Green,4 GB,64 GB,13MP + Depth Sensor,8MP Front Camera,17.32 cm (6.82 inch),1640 x 720 Pixels,MediaTek Helio G35,Octa Core,6000 mAh,"₹8,499"
1,Infinix,Hot 10 Play,Aegean Blue,4 GB,64 GB,13MP + Depth Sensor,8MP Front Camera,17.32 cm (6.82 inch),1640 x 720 Pixels,MediaTek Helio G35,Octa Core,6000 mAh,"₹8,499"
2,realme,C21,Cross Blue,4 GB,64 GB,13MP + 2MP + 2MP,5MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,MediaTek Helio G35,Octa Core,5000 mAh,"₹8,999"
3,realme,C21,Cross Black,3 GB,32 GB,13MP + 2MP + 2MP,5MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,MediaTek Helio G35,Octa Core,5000 mAh,"₹7,999"
4,realme,C21,Cross Black,4 GB,64 GB,13MP + 2MP + 2MP,5MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,MediaTek Helio G35,Octa Core,5000 mAh,"₹8,999"
5,realme,C21,Cross Blue,3 GB,32 GB,13MP + 2MP + 2MP,5MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,MediaTek Helio G35,Octa Core,5000 mAh,"₹7,999"
6,POCO,M3,Cool Blue,6 GB,64 GB,48MP + 2MP + 2MP,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 Pixels,Qualcomm Snapdragon 662,Octa Core,6000 mAh,"₹10,999"
7,REDMI,9i,Midnight Black,4 GB,64 GB,13MP Rear Camera,5MP Front Camera,16.59 cm (6.53 inch),1600 x 720 Pixels,MediaTek Helio G25,Octa Core,5000 mAh,"₹8,299"


##### 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps

In [83]:
driver=webdriver.Chrome('chromedriver.exe')
time.sleep(2)
driver.get('https://www.google.com/maps/')

City = input('Enter City Name : ')
driver.find_element_by_xpath("//input[@class='tactile-searchbox-input']").send_keys(City)
driver.find_element_by_xpath("//button[@class='searchbox-searchbutton']").click()

latitude=driver.current_url.split('/@')[1].split(',')[0]
longtitude=driver.current_url.split('/@')[1].split(',')[1].split(',')[0]
print('Latitude:',latitude,'Longitude:',longtitude)

Enter City Name : Surat
Latitude: 18.8154265 Longitude: 76.7751434


###### 6. Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 – September 20) from trak.in.

In [149]:
#loading the driver and the url
driver=webdriver.Chrome('chromedriver.exe')
driver.get('https://trak.in/')
time.sleep(2)

driver.get(driver.find_element_by_xpath('//li[@id="menu-item-51510"]/a').get_attribute('href'))

In [154]:
#creating the empty list 
date=[]
name=[]
industry=[]
sub=[]
city=[]
investor=[]
inv_type=[]
amount=[]

#scrapping for date
for i in driver.find_elements_by_xpath('//td[@class="column-2"]'):
    date.append(i.text)

#scrapping for name
for i in driver.find_elements_by_xpath('//td[@class="column-3"]'):
    name.append(i.text)

#scrapping for industry    
for i in driver.find_elements_by_xpath('//td[@class="column-4"]'):
    industry.append(i.text)

#scrapping for sub industry
for i in driver.find_elements_by_xpath('//td[@class="column-5"]'):
    sub.append(i.text)

#scrapping for city    
for i in driver.find_elements_by_xpath('//td[@class="column-6"]'):
    city.append(i.text)

#scrapping for investor
for i in driver.find_elements_by_xpath('//td[@class="column-7"]'):
    investor.append(i.text)

#scrapping for investor type    
for i in driver.find_elements_by_xpath('//td[@class="column-8"]'):
    inv_type.append(i.text)

#scrapping for amount
for i in driver.find_elements_by_xpath('//td[@class="column-9"]'):
    amount.append(i.text)

In [155]:
#checking for length of each columns
len(date),len(name),len(industry),len(sub),len(city),len(investor),len(inv_type),len(amount)

(392, 392, 392, 392, 392, 392, 392, 392)

In [157]:
#creating the dataframe for Jul-Sep
Funding={'Date':date,'Name':name,'Industry/Vertica':industry,'Sub-vertical':sub,'City':city,'Investor':investor,
        'Investment Type':inv_type,'Amount':amount}
Funding=pd.DataFrame(data=Funding)
Funding[55:85]

,Date,Name,Industry/Vertica,Sub-vertical,City,Investor,Investment Type,Amount
55,08/09/2020,Byju’s,EduTech,Online Tutoring,Bangalore,"Silver Lake, Tiger Global, General Atlantic an...",Private Equity,"500,000,000"
56,12/09/2020,mCaffeine,Personal Care,Skincare & Haircare,Mumbai,"Amicus Capital Private Equity I LLP, Amicus Ca...",Series B,"3,000,000"
57,09/09/2020,Qshala,EduTech,Online Curiosity Platform for Kids,Bangalore,Rainmatter Capital,Angel,"370,000"
58,02/09/2020,Winzo,Online Gaming,Online Gaming,New Delhi,"Kalaari Capital Partners, IndigoEdge Managemen...",Series B,"15,500,000"
59,09/09/2020,Hippo Video,Video Customer Experience(CX) Platform,Video Customer Experience(CX) Platform,"Newark, Delaware, United States of Amercia","Alpha Wave Incubation, Exfinity Venture Partne...",Series A,"4,500,000"
60,07/09/2020,Melorra,E-commerce,Online Jewelry Store,Bangalore,"Shadow Holdings, Lightbox.",Debt Financing,"upto 8,900,000"
61,07/09/2020,1mg,E-commerce,Online Pharmacy,Gurgaon,"Gaja Capital, Tata Capital, Partners Group",In Progress,"100,000,000"
62,31/08/2020,mfine,HealthTech,On-Demand Healthcare Services,Bangalore,Caretech Pte Inc,Series B,"5,400,000"
63,31/08/2020,Apna,Human Resources,Recruitment Platform,Bangalore,Lightspeed India and Sequoia Capital India,Series A,"8,000,000"
64,03/09/2020,Railofy,Transportation,WL & RAC protection platform,Mumbai,Chiratae Ventures,Seed,"950,000"


##### 7. Write a program to scrap all the available details of best gaming laptops from digit.in.

In [158]:
# loading the drivers and URL
driver=webdriver.Chrome('Chromedriver.exe')
driver.get('https://www.digit.in/')

In [159]:
#clicking on the Top10
driver.find_element_by_xpath('//div[@class="menu"]/ul/li[4]/a').click()
time.sleep(3)

In [160]:
#clicking on best gaming laptop in India
driver.find_element_by_xpath('//div[@class="listing_container"]/ul/li[26]/a').click()

In [166]:
#creating the empty list
name=[]
os=[]
display=[]
processor=[]
memory=[]
Weight=[]
Dimension=[]
graphic=[]

#scrapping for name
for i in driver.find_elements_by_xpath('//div[@class="TopNumbeHeading sticky-footer"]/a/h3'):
    name.append(i.text)

#scrapping for OS
for i in driver.find_elements_by_xpath('//div[@class="Spcs-details"]/table/tbody/tr[3]/td[3]'):
    os.append(i.text)

#scrapping for display
for i in driver.find_elements_by_xpath('//div[@class="Spcs-details"]/table/tbody/tr[4]/td[3]'):
    display.append(i.text)

#scrapping for processor type
for i in driver.find_elements_by_xpath('//div[@class="Spcs-details"]/table/tbody/tr[5]/td[3]'):
    processor.append(i.text)

#scrapping for memory
for i in driver.find_elements_by_xpath('//div[@class="Spcs-details"]/table/tbody/tr[6]/td[3]'):
    memory.append(i.text)
    
#scrapping for weight of the laptop
for i in driver.find_elements_by_xpath('//div[@class="Spcs-details"]/table/tbody/tr[7]/td[3]'):
    Weight.append(i.text)
    
#scrapping for dimension of the laptop
for i in driver.find_elements_by_xpath('//div[@class="Spcs-details"]/table/tbody/tr[8]/td[3]'):
    Dimension.append(i.text)
    
#scrapping for graphic processor
for i in driver.find_elements_by_xpath('//div[@class="Spcs-details"]/table/tbody/tr[9]/td[3]'):
    graphic.append(i.text)


In [177]:
#creating the dataframe
Top10={'Name':name,'OS':os[:6],'Display':display[:6],'Processor':processor[:6],'Memory':memory[:6],'Weight':Weight[:6],
      'Dimension':Dimension[:6],'Graphic Processor':graphic[:6]}
Top10=pd.DataFrame(data=Top10)
Top10

,Name,OS,Display,Processor,Memory,Weight,Dimension,Graphic Processor
0,ALIENWARE AREA 51M R2,Windows 10 Home,"17.3"" (1920 x 1080)",10th Generation Intel® Core™ i7-10700 | 2.90 GHz,1 TB SSD/16 GBGB DDR4,4.1,27.65 x 402.6 x 319.14,Intel® UHD Graphics 630
1,ASUS ROG STRIX SCAR 15,Windows 10 Home,"15.6"" (3840 x 2160)",10th Generation Intel® Core™ i9-10980HK | NA,1 TB SSD/16 GBGB DDR4,NA,NA,NA
2,ASUS ROG ZEPHYRUS G14,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 9 5900HX | 3.3 GHz,1 TB SSD/16 GBGB DDR4,2.30,35.4 x 25.9 x 2.26,NVIDIA® GeForce RTX™ 3070
3,LENOVO LEGION 5I,Windows 10 Home,"14"" (1920 x 1080)",AMD 3rd Generation Ryzen 9 | 3.3 GHz,1 TB SSD/16 GBGB DDR4,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060
4,ASUS ROG ZEPHYRUS DUO 15,Windows 10 Pro,"15.6"" (1920 x 1080)",10th Generation Intel® Core™ i5-10300H | 2.50 GHz,1 TB SSD/16 GBGB DDR4,2.3,363.06 x 259.61 x 23.57,NVIDIA® GeForce® GTX 1650 4GB
5,ACER ASPIRE 7 GAMING,Windows 10,"15.6"" (3840 x 1100)",Intel Core i7 10th Gen 10875H | NA,512 GB SSD/4 GBGB DDR4,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q


##### 8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [231]:
#loading the drivers and url
driver=webdriver.Chrome("chromedriver.exe") 
time.sleep(3)

driver.get('https://www.forbes.com')

driver.find_element_by_xpath("//button[@class='icon--hamburger']").click() # clicking on the hamburget button

#getting the billionaire page
billionaire=driver.find_element_by_xpath("//li[@class='header__channel header__color--centennial-silver header__hoverable']/a")
driver.get(billionaire.get_attribute('href'))
time.sleep(2)

driver.find_element_by_xpath("//h2[@class='chansec-special-feature__title no-label--special-feature']").click()
time.sleep(2)

#creating the empty list
Rank=[]
Name=[]
Net_worth=[]
Age=[]
Citizenship=[]
Source=[]
Industry=[]

#there are 15 pages hence we will scrap through range function
start=0
end=15
for page in range(start,end):
    #scrapping for rank
    rank=driver.find_elements_by_xpath("//div[@class='rank']")
    for i in rank:
        Rank.append(i.text)

    #scrapping for name
    name=driver.find_elements_by_xpath("//div[@class='personName']")
    for i in name:
        Name.append(i.text)

    #scrapping for Networth
    networth=driver.find_elements_by_xpath("//div[@class='netWorth']/div")
    for i in networth:
        Net_worth.append(i.text)
    
    #scrapping for age
    age=driver.find_elements_by_xpath("//div[@class='age']/div")
    for i in age:
        Age.append(i.text)
    
    #scrapping for citizenship
    citi=driver.find_elements_by_xpath("//div[@class='countryOfCitizenship']")
    for i in citi:
        Citizenship.append(i.text)

    #scrapping for source
    source=driver.find_elements_by_xpath("//div[@class='source-column']/div[1]")
    for i in source:
        Source.append(i.text)
    
    #scrapping for industry
    industry=driver.find_elements_by_xpath("//div[@class='category']/div")
    for i in industry:
        Industry.append(i.text)
    
    #to go on the next page
    try:
        nxt=driver.find_element_by_xpath("//button[@class='pagination-btn pagination-btn--next ']").click()
    except:
        NoSuchElementException


In [239]:
#creating the dataframe
Forbes_Billionarie={'Rank':Rank,'Name':Name,'Net Worth':Net_worth,'Age':Age,
                    'Citizenship':Citizenship,'Source':Source,'Industry':Industry}
Forbes_Billionarie=pd.DataFrame(data=Forbes_Billionarie)
Forbes_Billionarie

,Rank,Name,Net Worth,Age,Citizenship,Source,Industry
0,1.,Jeff Bezos,$177 B,,United States,Amazon,Technology
1,2.,Elon Musk,$151 B,,United States,"Tesla, SpaceX",Automotive
2,3.,Bernard Arnault & family,$150 B,,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$124 B,,United States,Microsoft,Technology
4,5.,Mark Zuckerberg,$97 B,,United States,Facebook,Technology
...,...,...,...,...,...,...,...
2905,2674.,Daniel Yong Zhang,$1 B,,China,e-commerce,Technology
2906,2674.,Zhang Yuqiang,$1 B,,China,Fiberglass,Manufacturing
2907,2674.,Zhao Meiguang,$1 B,,China,gold mining,Metals & Mining
2908,2674.,Zhong Naixiong,$1 B,,China,conglomerate,Diversified


##### 9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [20]:
#creating the empty list
Comments=[]
Upvote=[]
Updated=[]

#loading the drivers and url
driver=webdriver.Chrome("chromedriver.exe")
wait = WebDriverWait(driver,15)
driver.get("https://www.youtube.com/watch?v=ukzFI9rgwfU")

#creating the program to scroll down the comments and let it load
SCROLL_PAUSE_TIME = 2
CYCLES = 100

html = driver.find_element_by_tag_name('html')
html.send_keys(Keys.PAGE_DOWN)  
html.send_keys(Keys.PAGE_DOWN)  
time.sleep(SCROLL_PAUSE_TIME * 3)

for i in range(CYCLES):
    html.send_keys(Keys.END)
    time.sleep(SCROLL_PAUSE_TIME)

#scrapping for comments
comment = driver.find_elements_by_xpath('//*[@id="content-text"]')
for i in comment:
    Comments.append(i.text)

#scrapping for upvote
upvote = driver.find_elements_by_xpath('//*[@id="vote-count-middle"]')
for i in upvote:
    Upvote.append(i.text)

#scrapping for updated on
updated=driver.find_elements_by_xpath('//*[@id="header-author"]/yt-formatted-string/a')
for i in updated:
    Updated.append(i.text)

#creating the dataframe
Youtube={'Comments':Comments,'Comment Upvote':Upvote, 'Updated on':Updated}
Youtube=pd.DataFrame(data=Youtube)
Youtube

,Comments,Comment Upvote,Updated on
0,🔥 Enroll for FREE Machine Learning Course & Ge...,33,7 months ago (edited)
1,**Summary**:\n- machine learning is the genera...,2.5K,2 years ago
2,Literally learnt more from you than 4 years in...,57,1 month ago
3,I'm impressed by the way you taught. Teacher s...,189,9 months ago
4,wonderful and fantastic tutorial! It's really ...,21,2 years ago
...,...,...,...
576,1. Supervised\n\n2. Reinforcement --> Supervis...,,1 year ago
577,1.supervised\n2.reinforcement\n3.unsupervised,1,2 years ago
578,you should have use a better voice over withou...,1,1 year ago
579,1. Unsupervised\n2. Supervised \n3. Unsupervised,,2 years ago


##### 10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [219]:
#loading the driver and getting the url
driver=webdriver.Chrome("chromedriver.exe") 
time.sleep(2)
    
driver.get('https://www.hostelworld.com/')

In [220]:
#entering the city name in search bar
driver.find_element_by_xpath("//input[@id='search-input-field']").send_keys('London')

In [222]:
#selecting the city name from the dropdown
driver.find_element_by_xpath("//div[@class='label']").click()

In [223]:
#clicking on the search button
driver.find_element_by_xpath("//button[@id='search-button']").click()

In [225]:
#creating the empty list
Hostel_name=[]
Distance=[]
Rating=[]
Total_reviews=[]
Privates=[]
Dorms=[]
Facilities=[]
Property=[]
urls=[]

time.sleep(3)
#creating the for loop to scrap from 3 pages
start=0
end=3
for page in range(start,end):
    #scrapping for name of the hostel
    name=driver.find_elements_by_xpath("//h2[@class='title title-6']")
    for i in name:
        Hostel_name.append(i.text)
     
    #scrapping for the distance from the city
    distance=driver.find_elements_by_xpath("//span[@class='description']")
    for i in distance:
        Distance.append(i.text.split('-')[1].split('from')[0].replace(" ",''))
    
    #scrapping for the rating
    rating=driver.find_elements_by_xpath("//div[@class='keyword']")
    for i in rating:
        Rating.append(i.text)
    
    #scrapping for reviews
    reviews=driver.find_elements_by_xpath("//div[@class='reviews']")
    for i in reviews:
        Total_reviews.append(i.text.split(" ")[0])
    
    #scrapping for private price
    privates=driver.find_elements_by_xpath("//div[@class='price-col'][1]")
    for i in privates:
        Privates.append(i.text.split('\n')[1:])
        if (i.text.split('\n')[1:])is None:
            Privates.append('No Privates Available')
    
    #scrapping for dorms prices
    dorms=driver.find_elements_by_xpath("//div[@class='price-col'][2]")
    for i in dorms:
        Dorms.append(i.text.replace('\n',' '))
    
    #scrapping the url for each hostel
    url=driver.find_elements_by_xpath("//h2[@class='title title-6']/a")
    for i in url:
        urls.append(i.get_attribute('href'))
    try:
        nxt=driver.find_element_by_xpath("//div[@class='pagination-item pagination-next']").click()
    except:
        NoSuchElementException

#creating the for loop to scrap the facilities and description
for i in urls:
    driver.get(i)
    facilities=driver.find_elements_by_xpath("//ul[@class='facility-sections']")
    desc=driver.find_elements_by_xpath("//div[@class='content collapse-content']")
    for j in facilities:
        Facilities.append(j.text.replace('\n',' '))
    for k in desc:
        Property.append(k.text.replace('\n',' '))  

#creating the dataframce
Hostels={'Hostel Name':Hostel_name[:89],
'Distance':Distance,
'Rating': Rating[:89],
'Total Reviews':Total_reviews,
'Privates Price':Privates,
'Dorm Price':Dorms,
'Facilities':Facilities,
'Property':Property[:89]}

Hostels=pd.DataFrame(data=Hostels)
Hostels

,Hostel Name,Distance,Rating,Total Reviews,Privates Price,Dorm Price,Facilities,Property
0,St Christopher's Village,1.8km,Superb,10814,[Rs3567],Dorms From Rs1464,Free Linen Included Free City Maps Free WiFi F...,COVID 19 Policy Update. In response to Coronav...
1,Generator London,3km,NEW,6729,[Rs8540],Dorms From Rs2023,Free Linen Included Free City Maps Free WiFi F...,In probably one of the best locations for your...
2,Safestay London Kensington Holland Park,5.9km,Fabulous,1070,[],Dorms From Rs1019,Free Linen Included Free WiFi General Security...,Generator London is a design hotel-hostel loca...
3,"PubLove @ The Crown, Battersea",4.7km,Fabulous,204,[Rs7791],Dorms From Rs1317,Free Linen Included Free City Maps Free WiFi G...,Very good place to stay in London for any reas...
4,Leman House,3.6km,Very Good,13,[Rs6173],No Dorms Available,Free Free WiFi Free Internet Access General Br...,Safestay Holland Park Safestay Holland Park i...
...,...,...,...,...,...,...,...,...
84,The Dover,1.9km,No Rating,4,[Rs7202],No Dorms Available,Free Free Breakfast Free City Maps Towels Incl...,"Great location and good value. Free WiFi, coff..."
85,Park Hotel Essex,24.1km,Rating,108,[Rs3601],No Dorms Available,Free Free Breakfast Free Parking General Parki...,*Please note we are operating on LIMITED recep...
86,Cranbrook Hotel,14.8km,Fabulous,58,[Rs3601],No Dorms Available,Free Free Breakfast Free Parking General Hot S...,Celebrated my best friends birthday in London ...
87,St. Athans,2.9km,No Rating,234,[Rs3882],No Dorms Available,Free Linen Included Free City Maps Towels Incl...,Great hostel and can’t beat location either. S...


##### Completed